In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("data/IMDB Dataset.csv")
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
data = data.iloc[:15000]

In [5]:
data.drop_duplicates(inplace=True)

In [6]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [8]:
data['review'] = data['review'].apply(remove_tags)


In [9]:

data['review'] = data['review'].apply(lambda x:x.lower())

In [11]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

data['review'] = data['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

In [16]:

data['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [17]:
import gensim

In [18]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [20]:

story = []
for doc in data['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))
    

In [23]:
len(story)

158792

In [24]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [25]:
model.build_vocab(story)

In [26]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)


(8861217, 9333340)

In [27]:
len(model.wv.index_to_key)

38121

In [28]:

def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [30]:

document_vector(data['review'].values[0])


array([-0.15372643,  0.4731801 , -0.02900066,  0.12060585,  0.1189424 ,
       -0.31592458,  0.19678438,  0.5721752 , -0.3632937 , -0.10100759,
       -0.12782261, -0.44813964,  0.09264192,  0.42818305,  0.10865492,
       -0.25266853,  0.21074174, -0.46380866,  0.03547177, -0.6447462 ,
        0.13460709,  0.19754283, -0.03013981, -0.1046652 , -0.26841393,
       -0.13177665,  0.01253991, -0.08863675, -0.28246704,  0.04305064,
        0.37636063, -0.00215173, -0.10570794, -0.22121678, -0.21597199,
        0.43518168,  0.11925933, -0.42321688, -0.10333876, -0.69715774,
        0.00581603,  0.02652098,  0.02578342, -0.16551133,  0.473919  ,
       -0.10571051, -0.3233324 , -0.3427493 ,  0.1721631 ,  0.26524088,
        0.13211547, -0.26683038, -0.23731618, -0.15566306, -0.17476834,
        0.4216319 ,  0.4163248 ,  0.04175207, -0.34736675,  0.02705993,
       -0.16188827,  0.04596267, -0.13960451,  0.14933646, -0.22643274,
        0.24192022,  0.18885633,  0.13478935, -0.31066346,  0.40

In [33]:
data['review'].values[0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [34]:
from tqdm import tqdm

In [36]:

X = []
for doc in tqdm(data['review'].values):
    X.append(document_vector(doc))

100%|███████████████████████████████████████████████████████████████████████████████| 14961/14961 [07:45<00:00, 32.15it/s]


In [37]:
X = np.array(X)

In [39]:
X.shape

(14961, 100)

In [41]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(data['sentiment'])

In [42]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [44]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7888406281323087